# Descriptors

Descriptors specify what kinds of values are valid for a parameter.
There are currently, 2 variants: `RangeParameter` and `CategoryParameter`.  
(Note that even though their names end with Parameter, they are actually descriptors, not parameters.)

In [ ]:

from besos.parameters import (
    RangeParameter,
    CategoryParameter,
    Parameter,
)
from besos.problem import Problem
from besos import sampling
from besos.evaluator import EvaluatorGeneric

### RangeParameters
$min \leq x \leq max$

In [ ]:
zero_to_one_exclusive = RangeParameter(min_val=0.01, max_val=0.99)

### CategoryParameters
A list of options.

In [3]:
text_example = CategoryParameter(options=["a", "b", "c", "other"])
single_digit_integers = CategoryParameter(options=range(10))

### Sampling
These descriptors can be used to make `Parameters`.
Then we can generate samples.

In [4]:
parameters = [
    Parameter(value_descriptors=zero_to_one_exclusive, name="0-1"),
    Parameter(value_descriptors=single_digit_integers, name="single digit"),
    Parameter(value_descriptors=text_example, name="text"),
]
problem = Problem(parameters, outputs=["output"])

samples = sampling.dist_sampler(sampling.lhs, problem, num_samples=10)
samples

,0-1,single digit,text
0,0.636297,3,a
1,0.772004,7,a
2,0.534351,6,other
3,0.831262,5,a
4,0.088045,9,c
5,0.366748,1,b
6,0.218237,0,other
7,0.431348,2,c
8,0.138494,4,c
9,0.899764,8,b


### Evaluation
Since we did not specify selectors for the parameters, we cannot evaluate them using an EnergyPlus simulation.
Instead, we will use a custom evaluation function.

In [5]:
def evaluation_function(values):
    x, y, z = values
    if z == "other":
        return (x,)
    else:
        return (x * y,)


evaluator = EvaluatorGeneric(evaluation_function, problem)
# The evaluator will use this objective by default
outputs = evaluator.df_apply(samples, keep_input=True)
# outputs is a pandas dataframe with one column since only one objective was requested

In [6]:
outputs

,0-1,single digit,text,output
0,0.636297,3,a,1.908892
1,0.772004,7,a,5.404029
2,0.534351,6,other,0.534351
3,0.831262,5,a,4.156311
4,0.088045,9,c,0.792404
5,0.366748,1,b,0.366748
6,0.218237,0,other,0.218237
7,0.431348,2,c,0.862697
8,0.138494,4,c,0.553978
9,0.899764,8,b,7.198112
